In [ ]:
import logging
from typing import Any, Dict, List, Tuple

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine


def postgres_engine_factory(args: Dict[str, str]) -> Engine:
    """
    Create a PostgreSQL database engine from a dictionary of database info.
    """

    conn_string = f"postgresql://{args['POSTGRES_USER']}:{args['POSTGRES_PASSWORD']}@{args['POSTGRES_HOST']}:{args['POSTGRES_PORT']}/{args['POSTGRES_DATABASE']}"

    return create_engine(conn_string)


def execute_query(engine: Engine, query: str) -> List[Tuple[Any]]:
    """
    Execute DB queries safely using a PostgreSQL engine.
    """

    try:
        logging.warning(f"Running query on PostgreSQL: \n{query}")
        connection = engine.connect()
        results = connection.execute(query).fetchall()
    finally:
        connection.close()
        engine.dispose()
    return results


def execute_dataframe(engine: Engine, query: str) -> pd.DataFrame:
    """
    Execute a query on a PostgreSQL engine and return results as a DataFrame.
    """

    try:
        logging.warning(f"Running query on PostgreSQL: \n{query}")
        df = pd.read_sql_query(query, engine)
        return df
    except Exception as e:
        logging.error(f"Oh no! There was an error executing your query: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error


# Example usage:
args = {
    "POSTGRES_USER": "localhost",
    "POSTGRES_PASSWORD": "root",
    "POSTGRES_HOST": "localhost",
    "POSTGRES_PORT": "5432",
    "POSTGRES_DATABASE": "traffic",
}

# Create a PostgreSQL engine
engine = postgres_engine_factory(args)

# Execute a query and fetch results as a list of tuples
results = execute_query(engine, "SELECT * FROM your_table")

# Execute a query and fetch results as a DataFrame
df = execute_dataframe(engine, "SELECT * FROM your_table")

# Close the connection and dispose of the engine
engine.dispose()